In [11]:
import numpy as np
import pandas as pd
import statsmodels as sm
from plotnine import *
np.random.seed(12345)
from statsmodels.base.model import GenericLikelihoodModel

## Задание 1

In [40]:
#загрузим данные
data = pd.read_excel('loanapp.xls', header = None, 
                     names = ['Col' + str(x) for x in range(59)])
#удалим отсутствующие наблюдения
data = data.loc[(data['Col49']!='.') & (data['Col58']!='.') 
                 & (data['Col25']!='.') & (data['Col26']!='.') 
                 & (data['Col56']!='.') & (data['Col38']!='.') 
                 & (data['Col47']!='.') & (data['Col8']!='.')
                 & (data['Col9']!='.') & (data['Col44']!='.')
                 & (data['Col34']!='.') & (data['Col54']!='.')
                 & (data['Col24']!='.') & (data['Col52']!='.')
                 & (data['Col53']!='.') & (data['Col43']!='.')]
#выберем колонки для анализа
data_estimation = pd.DataFrame({'Intercept':[1]*len(data),
                                'approve': data['Col49'], #whether loan was approved or not
                                'white': data['Col58'], #1 if applicant is white
                                'hrat': data['Col25'], #housing expenditure, % of total income
                                'obrat': data['Col26'], #other obligations, % of total income
                                'loanprc': data['Col56'], #loan amount / the total price
                                'unem': data['Col38'], #unemployment rate by industry
                                'male': data['Col47'], #1 if applicant is male
                                'married': data['Col8'], #1 if applicant is married
                                'dep': data['Col9'], #number of dependents 
                                'sch': data['Col44'], #1 if more than 12 years of schooling
                                'cosign': data['Col34'], #1 if there is a cosigner on the loan
                                'chist': data['Col54'], #1 credit history (0 if there are delinquent accounts)
                                'pubrec': data['Col24'], #1 if ever filed for bankruptcy 
                                'mortlat1': data['Col52'], #1 if 1 or 2 late mortgage payments
                                'mortlat2': data['Col53'], #1 if more than 2 late mortgage payments 
                                'vr': data['Col43']})
data_estimation = data_estimation.astype(float)
#оценим модель 
model = sm.discrete.discrete_model.Logit.from_formula('approve ~ white + hrat + obrat + loanprc + unem + male + married + dep + sch + cosign + chist + pubrec + mortlat1 + mortlat2 + vr', 
                                                      data = data_estimation)
results = model.fit(method = 'bfgs', maxiter = 1000)
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.304666
         Iterations: 87
         Function evaluations: 90
         Gradient evaluations: 90
                           Logit Regression Results                           
Dep. Variable:                approve   No. Observations:                 1971
Model:                          Logit   Df Residuals:                     1955
Method:                           MLE   Df Model:                           15
Date:                Sat, 09 Mar 2024   Pseudo R-squ.:                  0.1863
Time:                        14:27:25   Log-Likelihood:                -600.50
converged:                       True   LL-Null:                       -737.98
Covariance Type:            nonrobust   LLR p-value:                 8.693e-50
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.8014      0.595      

1. Какая переменная имеет наибольшее влияние (по модулю) на одобрение займа? 
2. Интерпретируйте эффект переменной loanprc на одобрение займа, используя следующий алгоритм:

    а) сгенерируйте 1000 возможных коэффициентов регрессии, используя генератор случайных чисел для многомерного нормального распределения со средним = оцененным параметрам и ковариционной матрицей, полученной с помощью метода cov_params()
    
    б) для каждой строчки в массиве и возможного вектора коэффициентов регрессии из части а посчитайте частную производную логит линк-функции относительно loanprc
    
    в) посчитайте среднее арифметическое по всем возможным частным производным, а такжу 2.5 и 97.5 перцентили получившегося распределения частных производных (см. формат вывода в следующей ячейке, нужно заменить числа на правильные). Какой вывод можно сделать?

3. Перечислите переменные, эффект которых значим при $\alpha$ = 0.001, двусторонний тест.

In [3]:
print("Effect Estimate:", 0.3, "****", "Confidence Interval:", [0.1,0.5])

Effect Estimate: 0.3 **** Confidence Interval: [0.1, 0.5]


4. Используя подход log-odds, проиллюстрируйте эффект переменной obrat на вероятность одобрения займа.
5. Используя подход end point transformation, проиллюстрируйте эффект переменной obrat на вероятность одобрения займа. 
6. Перед решением этого вопроса сделайте задание 2! Уберите переменную white из модели и протестируйте нулевую гипотезу о достаточно модели без переменной white, используя тест отношения правдоподобия (likelihood ratio test). Какой вывод можно сделать? 
7.  Перед решением этого вопроса сделайте задание 2! Проделайте то же самое, но теперь вместо white уберите переменную mortlat2. Какой вывод можно сделать? 

## Задание 2

In [43]:
#загрузим данные по криминальной статистике в США (пояснения см. в txt файле)
#единица анализа - индивид 
#загрузим данные
data = pd.read_excel('crime1.xls', header = None, 
                     names = ['Col' + str(x) for x in range(16)])
#удалим отсутствующие наблюдения
data = data.loc[(data['Col0']!='.') & (data['Col1']!='.') 
                 & (data['Col2']!='.') & (data['Col3']!='.') 
                 & (data['Col4']!='.') & (data['Col5']!='.') 
                 & (data['Col6']!='.') & (data['Col7']!='.')
                 & (data['Col8']!='.') & (data['Col9']!='.')
                 & (data['Col10']!='.') & (data['Col11']!='.')
                 & (data['Col12']!='.') & (data['Col13']!='.')
                 & (data['Col14']!='.') & (data['Col15']!='.')]
#выберем колонки для анализа
data_estimation = pd.DataFrame({'Intercept':[1]*len(data),
                                'narr86': data['Col0'], # times arrested, 1986
                                'nfarr86': data['Col1'], # felony arrests, 1986
                                'nparr86': data['Col2'], # property crme arr., 1986
                                'pcnv': data['Col3'], #proportion of prior convictions
                                'avgsen': data['Col4'], # avg sentence length, mos.
                                'tottime': data['Col5'], #time in prison since 18 (mos.)
                                'ptime86': data['Col6'], #mos. in prison during 1986
                                'qemp86': data['Col7'], # quarters employed, 1986
                                'inc86': data['Col8'], #legal income, 1986, $100s
                                'durat': data['Col9'], # recent unemp duration
                                'black': data['Col10'], # =1 if black
                                'hispan': data['Col11'], #=1 if Hispanic
                                'born60': data['Col12'], #=1 if born in 1960
                                'pcnvsq': data['Col13'], #pcnv^2
                                'pt86sq': data['Col14'], # ptime86^2
                                'inc86sq': data['Col15']}) #inc86^2
data_estimation = data_estimation.astype(float)
#саммари статистики 
data_estimation.describe(include = 'all')

,Intercept,narr86,nfarr86,nparr86,pcnv,avgsen,tottime,ptime86,qemp86,inc86,durat,black,hispan,born60,pcnvsq,pt86sq,inc86sq
count,2725.0,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000,2725.000000
mean,1.0,0.404404,0.233394,0.125505,0.357787,0.632294,0.838752,0.387156,2.309028,54.967046,2.251376,0.161101,0.217615,0.362569,0.284131,3.951193,7458.932607
std,0.0,0.859077,0.581014,0.482847,0.395192,3.508031,4.607019,1.950051,1.610428,66.627213,4.607063,0.367691,0.412700,0.480830,0.390734,22.085840,16361.238469
min,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.160000
50%,1.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,3.000000,29.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,841.000000
75%,1.0,1.000000,0.000000,0.000000,0.670000,0.000000,0.000000,0.000000,4.000000,90.100000,2.000000,0.000000,0.000000,1.000000,0.448900,0.000000,8118.010000
max,1.0,12.000000,6.000000,8.000000,1.000000,59.200000,63.400000,12.000000,4.000000,541.000000,25.000000,1.000000,1.000000,1.000000,1.000000,144.000000,292681.000000


In [46]:
#оценим Пуассоновскую регрессию 
model_p = sm.discrete.discrete_model.Poisson.from_formula('narr86 ~ pcnv + avgsen + tottime + ptime86 + qemp86 + inc86 + black + hispan + born60', 
                                                      data = data_estimation)
results_p = model_p.fit()
print(results_p.summary())
#сохраним результаты для теста отношения правдоподобий
log_likelihood_full = results_p.llf

Optimization terminated successfully.
         Current function value: 0.825233
         Iterations 6
                          Poisson Regression Results                          
Dep. Variable:                 narr86   No. Observations:                 2725
Model:                        Poisson   Df Residuals:                     2715
Method:                           MLE   Df Model:                            9
Date:                Sat, 09 Mar 2024   Pseudo R-squ.:                 0.07910
Time:                        14:44:02   Log-Likelihood:                -2248.8
converged:                       True   LL-Null:                       -2441.9
Covariance Type:            nonrobust   LLR p-value:                 1.134e-77
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.5996      0.067     -8.916      0.000      -0.731      -0.468
pcnv          -0.4016      0.

In [47]:
#оценим Негативную Биномиальную регрессию
model_n = sm.discrete.discrete_model.NegativeBinomial.from_formula('narr86 ~ pcnv + avgsen + tottime + ptime86 + qemp86 + inc86 + black + hispan + born60', 
                                                      data = data_estimation)
results_n = model_n.fit()
print(results_n.summary())
#сохраним результаты для теста отношения правдоподобий
log_likelihood_nb = results_n.llf

Optimization terminated successfully.
         Current function value: 0.791790
         Iterations: 32
         Function evaluations: 38
         Gradient evaluations: 38
                     NegativeBinomial Regression Results                      
Dep. Variable:                 narr86   No. Observations:                 2725
Model:               NegativeBinomial   Df Residuals:                     2715
Method:                           MLE   Df Model:                            9
Date:                Sat, 09 Mar 2024   Pseudo R-squ.:                 0.05809
Time:                        14:44:09   Log-Likelihood:                -2157.6
converged:                       True   LL-Null:                       -2290.7
Covariance Type:            nonrobust   LLR p-value:                 3.907e-52
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.5638      0.083     -

1. Обоснуйте содержательно включение каждой из переменных в регрессии выше. 

2. Используя сохранёные значения log likelihood и тест отношения правдоподобия (likelihood ratio), оцените, является ли Пуассоновская модель достаточной? Присутствует ли в даннных overdispersion? 

3. См. ячейку ниже. 

In [ ]:
#используйте CLARIFY для оценки эффекта дохода на число арестов 
variance_covariance = results_p.cov_params()
#сгенерируйте матрицу возможных регрессионных коэффициентов из многомерного нормального распределения 
#с вектором средних в качестве means и variance-covariance в качестве ковариационной матрицы
#количество строк матрицы - 1000, используйте генератор случайных чисел из numpy 
coef_distribution = #ВАШ КОД ЗДЕСЬ
binary_variables = ['black', 'hispan', 'born60']
continuous_variables = ['pcnv', 'avgsen', 'tottime', 'ptime86', 'qemp86']
#установим предикторы кроме дохода на постоянные значения
data_plot = data_estimation[['Intercept', 'pcnv', 'avgsen', 'tottime', 'ptime86', 'qemp86', 'inc86', 'black', 'hispan', 'born60']].copy()
for x in binary_variables:
    mean_value = np.mean(data_plot[x])
    if mean_value >=0.5:
        data_plot[x] = 1
    else:
        data_plot[x] = 0
for x in continuous_variables:
    data_plot[x] = np.mean(data_plot[x])
#умножим матрицы
matrix_score = np.exp(np.dot(data_plot, np.transpose(coef_distribution)))
predicted_arrests = np.mean(matrix_score, axis = 1)
upper_bound = np.percentile(matrix_score, axis = 1, q = 97.5)
lower_bound = np.percentile(matrix_score, axis = 1, q = 2.5)
#постройте график, где по оси абсцисс отложены значения дохода, а по оси ординат - ожидаемое количество арестов
#ВАШ КОД ЗДЕСЬ 

4. Напишите код для оценки Пуассоновской регрессии с помощью функционала библиотеки statsmodels (см. ячейку ниже)

In [ ]:
class MyPoisson(GenericLikelihoodModel):
    #напишите функцию, возвращающую для массива значение log-likelihood 
    def loglike(self, params):
        #exog - матрица предикторов (независимых переменных); endog - вектор значений зависимой переменной
        exog = self.exog
        endog = self.endog
        
        #ВАШ КОД ЗДЕСЬ
        
        return log_likelihood 
sm_poisson_manual = MyPoisson.from_formula('narr86 ~ pcnv + avgsen + tottime + ptime86 + qemp86 + inc86 + black + hispan + born60', 
                                        data = data_estimation).fit(method = 'lbfgs', maxiter = 1000, start_params = [0.01]*10)
#сравните Ваши результаты с результатами модели, оцененной в начале задания statsmodels
print(sm_poisson_manual.summary())

5. Используя алгоритм, описанный в первом задании, вопрос 2, оцените эффект переменной black на количество арестов. Какой вывод можно сделать?

6. Используя модель с инфляцией нулей в ячейке ниже, ответьте на следующие вопросы:

а) оправдалась ли гипотеза о влиянии указанных в ячейке ниже переменных на **вероятность** полицейского внимания? (обоснуйте ответ)

б) в целом, является ли модель с инфляцией необходимой для данной задачи? 


In [ ]:
#Пуассоновская модель с инфляцией нулей
#suppose the data-generating process works like this: with probability p_i police does not pay attention to a
#person, meaning that some of the illegal actions remained unnoticed, while with probability (1 - p_i)
#police does pay attention to a person
#p_i, therefore, is the probability that police does not scrutinize a certain person
#while 1 - p_i is the probability that police does scrutinize a certain person
#we can expect that the probability of police scrutiny is bigger for blacks and hispanics
#on the other hand, the probability of police scrutiny is likely lower for richer people
#this situation requires zero-inflated modelling 
#run the zero-inflated model 
exog = data_estimation[['Intercept', 'pcnv', 'avgsen', 'tottime', 'ptime86', 'qemp86', 'inc86', 'black', 'hispan', 'born60']]
exog_inflation = data_estimation[['Intercept', 'inc86', 'black', 'hispan']]
model = sm.discrete.count_model.ZeroInflatedPoisson(data_estimation['narr86'],
                                                    exog = exog,
                                                    exog_infl = exog_inflation)
results_infl = model.fit(maxiter = 200)
print(results_infl.summary())